In [2]:
import numpy as np
import pandas as pd

# plot
import matplotlib.pyplot as plt

# tf
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import splitfolders

## Data Collecting

Dataset diambil dari website kaggle.

### Tujuan PA

Membuat sebuah model yang dapat mengenali jenis dari serangga dengan menggunakan algoritma CNN.

Batasan:
  - Model ini hanya dapat mengenali:
    - Kupu - Kupu, 
    - Capung, 
    - Nyamuk, 
    - Belalang,
    - dan Kumbang.

### Deskripsi Dataset

Dataset ini berupa gambar dari 5 serangga berbeda.

### Jumlah Dataset

- Kupu - Kupu: 899
- Capung: 1036
- Nyamuk: 690
- Belalang: 960
- Kumbang: 864

Total: 4449 data

### Data Train & Validation

URL path ke direktori dataset.
berupa direktori dataset.
dan dataset output (dataset yang akan dihasilkan ketika melakukan split folder). 

In [4]:
url = 'dataset_serangga'
url_split = 'dataset-output'

In [5]:
splitfolders.ratio(url, output=url_split, seed=42, ratio=(0.7,0.2,0.1), group_prefix=None)

Disini saya menggunakan Image data generator dan batch size sebanyak 128
untuk validation, saya menggunakan 20% data.
Kemudian untuk subset training dan validation, target size nya adalah 120.

ambil path dari folder masing - masing output

In [6]:
train = 'dataset-output/train'
val = 'dataset-output/val'
test = 'dataset-output/test'

In [7]:
batch_size = 128

train_image_generator = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=45,
    horizontal_flip=True,
)

train = train_image_generator.flow_from_directory(
    train, 
    target_size=(120,120),
    batch_size=batch_size, 
)

test = train_image_generator.flow_from_directory(
    test, 
    target_size=(120,120),
    batch_size=batch_size,
)

val = train_image_generator.flow_from_directory(
    val, 
    target_size=(120,120),
    batch_size=batch_size,
)

Found 3112 images belonging to 5 classes.
Found 449 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


# Model 

Pada syntax di atas model dibentuk dengan suatu layer convo 2 dimensi dari size 120 x 120 pixels dimana fungsi aktivasinya adalah. Terdapat dua pembuatan layer convo, sehingga dilanjutkan dengan pembuatan layer max pooling dan pembuatan layer dropout. Karena ini adalah pelatihan deep learning maka perlu pelatihan lagi dengan membuat lagi layer untuk pelatihan. Kemudian dilanjutkan dengan layer flatten dan layer dense sampai dengan proses kompilasi.



In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        64, 
        (3,3), 
        activation='relu',
        input_shape=(120, 120, 3), 
        padding='same'
        ),

      tf.keras.layers.Conv2D(
        64, (3,3), 
        activation='relu',
        input_shape=(120, 120, 3),
        padding='same'
        ),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(
        256, 
        (3,3), 
        activation='relu', 
        padding='same'
        ),
    
    tf.keras.layers.Conv2D(
      256, 
      (3,3), 
      activation='relu',
      padding='same'
      ),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),
    

    tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(1024, activation='relu'),
    # tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')     
])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 60, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 60, 60, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 256)       147712    
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 256)       590080    
                                                        

In [10]:
steps, val_steps = train.n/batch_size, val.n/batch_size
num_epochs = 50

epoch dibuat sebanyak 50

In [11]:
history = model.fit(train, validation_data=val, epochs=num_epochs,
                    steps_per_epoch=steps, validation_steps=val_steps)

Epoch 1/50
24/24 [==============================] - 1337s 51s/step - loss: 1.9946 - accuracy: 0.2629 - val_loss: 1.4878 - val_accuracy: 0.3761
Epoch 2/50
24/24 [==============================] - 1411s 59s/step - loss: 1.4717 - accuracy: 0.3406 - val_loss: 1.4354 - val_accuracy: 0.3694
Epoch 3/50
24/24 [==============================] - 1174s 48s/step - loss: 1.3955 - accuracy: 0.3904 - val_loss: 1.3527 - val_accuracy: 0.3851
Epoch 4/50
24/24 [==============================] - 1104s 45s/step - loss: 1.3601 - accuracy: 0.4145 - val_loss: 1.2998 - val_accuracy: 0.4955
Epoch 5/50
24/24 [==============================] - 993s 41s/step - loss: 1.3063 - accuracy: 0.4534 - val_loss: 1.2159 - val_accuracy: 0.5146
Epoch 6/50
24/24 [============================>.] - ETA: 16s - loss: 1.2451 - accuracy: 0.4953 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Akurasi Training')
plt.plot(epochs, val_acc, 'g', label='Akurasi Validation')

plt.xlabel('Epochs')
plt.ylabel('Akurasi')
plt.title('Akurasi Training and validation')
plt.grid(axis='both')

plt.show() 